In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import statsmodels.api as sm
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error



In [2]:


# Specify the file path
file_path = 'raw_data/consolidado.csv'

# Read the CSV file into a DataFrame+
df = pd.read_csv(file_path)

df['price_soybean'] = df['price_soybean'].str.replace(',', '.')
# Convert 'date' column to datetime
df['date'] = pd.to_datetime(df['date'])
# Filter out data from the year 2023
df= df[df['date'].dt.year != 2023]

df['price_soybean'] = df['price_soybean'].str.replace(',', '.')
df['price_soybean'] = pd.to_numeric(df['price_soybean'], errors='coerce')


selected_columns = ['date','price_soybean','real_interest_rate',
                    'SP500',
                    'SOYBEANS - USA PRODUCTION [mTons]']
df=df[selected_columns]





In [3]:

#features seleccionadas
df = df[['date','price_soybean', 'real_interest_rate', 'SP500', 'SOYBEANS - USA PRODUCTION [mTons]']]

selected_columns=['real_interest_rate', 'SP500', 'SOYBEANS - USA PRODUCTION [mTons]']


#Limpieza general de datos

# Convert 'date' column to datetime
df['date'] = pd.to_datetime(df['date'])
# Sacamos el año 2023
df= df[df['date'].dt.year != 2023]

# Convertimos el precio de la soja a numerico
df['price_soybean'] = pd.to_numeric(df['price_soybean'], errors='coerce')



#Creamos las nuevas columnas t-1,t-3,t-6; agregamos medias moviles del precio de la soja 3m,6m,12m
df['previous_price_soybean'] = df['price_soybean'].shift(1)
df['previous_usa_production'] = df['SOYBEANS - USA PRODUCTION [mTons]'].shift(6)
df['previous_SP500'] = df['SP500'].shift(1)
df['previous_real_interest_rate'] = df['real_interest_rate'].shift(1)
df['price_soybean_avg_3_months'] = df['price_soybean'].rolling(window=3).mean()
df['price_soybean_avg_6_months'] = df['price_soybean'].rolling(window=6).mean()
df['price_soybean_avg_12_months'] = df['price_soybean'].rolling(window=12).mean()
df['3mprevious_price_soybean'] = df['price_soybean'].shift(3)
df['3mprevious_SP500'] = df['SP500'].shift(3)
df['3mprevious_real_interest_rate'] = df['real_interest_rate'].shift(6)
df['6mprevious_price_soybean'] = df['price_soybean'].shift(6)
df['6mprevious_SP500'] = df['SP500'].shift(6)
df['6mprevious_real_interest_rate'] = df['real_interest_rate'].shift(6)
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year
df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
df['year_sin'] = np.sin(2 * np.pi * df['year'] / df['year'].max())
df['year_cos'] = np.cos(2 * np.pi * df['year'] / df['year'].max())


#si hay nulls mediana de los ultimos 12 meses
rolling_median = df.rolling(window=12, min_periods=1).median()
df_filled = df.fillna(rolling_median)
df = df_filled
df2 = df.fillna(method='bfill')
df=df2

#Scaler
scaler = MinMaxScaler()
df[selected_columns] = scaler.fit_transform(df[selected_columns])


#nos aseguramos que los nulls esten cubiertos
null_rows = df.isnull().sum(axis=1)
num_null_rows = null_rows.sum()



/tmp/ipykernel_685/3637742592.py:42: FutureWarning: Dropping of nuisance columns in rolling operations is deprecated; in a future version this will raise TypeError. Select only valid columns before calling the operation. Dropped columns were Index(['date'], dtype='object')
  rolling_median = df.rolling(window=12, min_periods=1).median()


In [4]:
df = df.rename(columns={'SOYBEANS - USA PRODUCTION [mTons]': 'usa_prod'})

print(f"Number of rows with null values: {num_null_rows}")


Number of rows with null values: 0


In [5]:
print(df.columns)


Index(['date', 'price_soybean', 'real_interest_rate', 'SP500', 'usa_prod',
       'previous_price_soybean', 'previous_usa_production', 'previous_SP500',
       'previous_real_interest_rate', 'price_soybean_avg_3_months',
       'price_soybean_avg_6_months', 'price_soybean_avg_12_months',
       '3mprevious_price_soybean', '3mprevious_SP500',
       '3mprevious_real_interest_rate', '6mprevious_price_soybean',
       '6mprevious_SP500', '6mprevious_real_interest_rate', 'month', 'year',
       'month_sin', 'month_cos', 'year_sin', 'year_cos'],
      dtype='object')


## Pasamos todo a INT, salvo date

In [6]:
# Select non-numeric columns
non_numeric_columns = df.select_dtypes(exclude=['float64', 'int64']).columns

# Print the non-numeric columns
print("Non-Numeric Columns:")
for column in non_numeric_columns:
    print(column)


Non-Numeric Columns:
date


In [27]:
#OLS

import statsmodels.api as sm

# Separate the target variable and the predictors
y = df['price_soybean']
X = df.drop(['price_soybean', 'date'], axis=1)  # Pass the columns to drop as a list

# Add a constant column to the predictors for the intercept term
X = sm.add_constant(X)

# Fit the linear regression model
model = sm.OLS(y, X)
results = model.fit()

# Print the regression summary
print(results.summary())

# Extract the p-values
p_values = results.pvalues

# Filter the features based on the significance level (e.g., p-value < 0.05)
significant_features = p_values[p_values < 0.05]

# Print the significant features
print("Significant Features:")
for feature, p_value in significant_features.iteritems():
    print(f"{feature}: p-value = {p_value:.4f}")


                            OLS Regression Results                            
Dep. Variable:          price_soybean   R-squared:                       0.985
Model:                            OLS   Adj. R-squared:                  0.984
Method:                 Least Squares   F-statistic:                     1236.
Date:                Tue, 04 Jul 2023   Prob (F-statistic):               0.00
Time:                        21:52:37   Log-Likelihood:                -2102.8
No. Observations:                 414   AIC:                             4250.
Df Residuals:                     392   BIC:                             4338.
Df Model:                          21                                         
Covariance Type:            nonrobust                                         
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
const 

In [30]:
import statsmodels.api as sm

# Separate the target variable and the predictors
y = df['price_soybean']
X = df.drop(['price_soybean', 'date'], axis=1)  # Pass the columns to drop as a list

# Fit the SARIMA model
model = sm.tsa.SARIMAX(y, exog=X, order=(1, 0, 1), seasonal_order=(1, 0, 1, 12))  # Modify the order as needed
results = model.fit()

# Print the model summary
print(results.summary())

# Get the p-values of the coefficients
p_values = results.pvalues

# Filter the features with statistically significant coefficients
significant_features = X.columns[p_values < 0.05]

# Print the significant features
print("Significant Features:")
for feature in significant_features:
    print(feature)


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           27     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  5.07775D+00    |proj g|=  1.76501D-02


 This problem is unconstrained.



At iterate    5    f=  5.07770D+00    |proj g|=  1.16433D-01

At iterate   10    f=  5.07678D+00    |proj g|=  1.00364D-01

At iterate   15    f=  5.07592D+00    |proj g|=  8.06999D-02

At iterate   20    f=  5.04803D+00    |proj g|=  1.89248D+00

At iterate   25    f=  5.03330D+00    |proj g|=  2.85859D+00

At iterate   30    f=  4.99933D+00    |proj g|=  2.34961D-01

At iterate   35    f=  4.99150D+00    |proj g|=  2.30252D-01

At iterate   40    f=  4.96046D+00    |proj g|=  1.64325D+00

At iterate   45    f=  4.94953D+00    |proj g|=  2.35840D-01


/home/pipeposse/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "



At iterate   50    f=  4.93770D+00    |proj g|=  3.13724D-01

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   27     50     63      1     0     0   3.137D-01   4.938D+00
  F =   4.9377017918197694     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
                                     SARIMAX Results                                      
Dep. Variable:                      price_soybean   No. Observations:                  414
Model:             SARIMAX(1, 0, 1)x(1, 0, 1, 12)   Log Likelihood               -2044.209
Date:                            Tue, 04 Jul 2023   AIC                           4142.4

IndexError: boolean index did not match indexed array along dimension 0; dimension is 22 but corresponding boolean dimension is 27

In [31]:
# Assuming 'results' contains the fitted SARIMAX model results

# Extract the coefficient values and confidence intervals for the exogenous variables
exog_coefficients = results.params.iloc[1:]
exog_ci = results.conf_int().iloc[1:]

# Combine the coefficient values and confidence intervals into a DataFrame
exog_summary = pd.concat([exog_coefficients, exog_ci], axis=1)
exog_summary.columns = ['Coefficient', 'Lower CI', 'Upper CI']

# Print the significance of each variable based on confidence intervals
print("Variable Significance based on Confidence Intervals:")
print(exog_summary)


Variable Significance based on Confidence Intervals:
                                   Coefficient      Lower CI      Upper CI
SP500                                71.393900    -54.590013    197.377813
SOYBEANS - USA PRODUCTION [mTons]   -23.093916   -152.269368    106.081536
previous_price_soybean               -0.591586     -0.834789     -0.348384
previous_usa_production             -33.164033   -148.046205     81.718138
previous_SP500                      -13.749861   -232.129717    204.629996
previous_real_interest_rate         -25.847935   -136.597381     84.901510
price_soybean_avg_3_months            1.910908      1.671546      2.150270
price_soybean_avg_6_months           -0.389173     -0.825901      0.047556
price_soybean_avg_12_months          -0.035828     -0.300106      0.228449
3mprevious_price_soybean              0.155902     -0.053067      0.364871
3mprevious_SP500                     -0.552253   -206.133129    205.028623
3mprevious_real_interest_rate        19.792671 

In [32]:
# Assuming 'results' contains the fitted SARIMAX model results

# Extract the z-values
z_values = results.tvalues

# Filter the features based on significance level (e.g., z-value > 1.96 for 95% confidence)
significant_features = z_values[abs(z_values) > 1.96]  # Modify the threshold as needed

# Print the significant features along with their z-values
print("Significant Features:")
for feature, z_value in significant_features.iteritems():
    print(f"{feature}: z-value = {z_value}")
    


Significant Features:
previous_price_soybean: z-value = -4.767587528992919
price_soybean_avg_3_months: z-value = 15.647056233613029
6mprevious_price_soybean: z-value = -2.1933525431345067
ma.L1: z-value = 7.261603818603921
sigma2: z-value = 11.18644857158151


## Vamos Con Lasso

In [37]:
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Separate the target variable and the predictors
y = df['price_soybean']
X = df.drop(['price_soybean', 'date'], axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and fit the Lasso model
lasso = Lasso(alpha=0.1)  # You can adjust the alpha parameter as needed
lasso.fit(X_train, y_train)

# Make predictions
y_pred = lasso.predict(X_test)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)
print("MAE: {:.2f}".format(mae))

# Calculate MAPE
mape = 100 * (mae / y_test.mean())
print("MAPE: {:.2f}%".format(mape))


MAE: 25.55
MAPE: 3.11%


/home/pipeposse/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.134e+05, tolerance: 3.346e+03
  model = cd_fast.enet_coordinate_descent(


In [6]:

import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

# Separate the target variable and the predictors
y = df['price_soybean']
X = df.drop(['price_soybean', 'date'], axis=1)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the XGBoost model
model = xgb.XGBRegressor()

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_pred = model.predict(X_test)

# Calculate MAE
mae = mean_absolute_error(y_test, y_pred)
print("MAE: {:.2f}".format(mae))

# Calculate MAPE
mape = 100 * (mae / y_test.mean())
print("MAPE: {:.2f}%".format(mape))


MAE: 31.40
MAPE: 3.82%


In [39]:
columns_with_special_chars = [col for col in df.columns if any(char in col for char in ['[', ']', '<'])]
print(columns_with_special_chars)


['SOYBEANS - USA PRODUCTION [mTons]']


In [13]:
#Random Forest

# Assuming 'df' is your DataFrame containing the selected features and target variable

# Separate the target variable and the predictors
y = df['price_soybean']
X = df.drop(['price_soybean','date'], axis=1)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the Random Forest model
model = RandomForestRegressor()

# Train the model
model.fit(X_train, y_train)


# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate Mean Absolute Error (MAE)
mae = mean_absolute_error(y_test, y_pred)
print("MAE:", mae)

# Calculate Mean Absolute Percentage Error (MAPE)
mape = 100 * (mae / y_test.mean())
print("MAPE:", mape)



MAE: 31.318433734939752
MAPE: 3.8107545482532643


In [11]:
import pandas as pd

# Assuming 'df' is your DataFrame

# Save the DataFrame as a CSV file
df.to_csv('output.csv', index=False)


In [14]:
column_names = df.columns.tolist()
print(column_names)


['date', 'price_soybean', 'real_interest_rate', 'SP500', 'usa_prod', 'previous_price_soybean', 'previous_usa_production', 'previous_SP500', 'previous_real_interest_rate', 'price_soybean_avg_3_months', 'price_soybean_avg_6_months', 'price_soybean_avg_12_months', '3mprevious_price_soybean', '3mprevious_SP500', '3mprevious_real_interest_rate', '6mprevious_price_soybean', '6mprevious_SP500', '6mprevious_real_interest_rate', 'month', 'year', 'month_sin', 'month_cos', 'year_sin', 'year_cos']
